# 1. Introducción a DataFrames:
## Conceptos básicos:
Los DataFrames son una abstracción de datos estructurados, organizados en filas y columnas, con un esquema definido. Esta estructura facilita la manipulación y el análisis de datos utilizando las APIs de Spark SQL.

## Creación de DataFrames:
- **Desde RDDs**: Los DataFrames pueden crearse a partir de RDDs (colecciones distribuidas de datos). La creación de un DataFrame desde un RDD permite trabajar con datos no estructurados transformándolos en un formato tabular.
- **Desde archivos**: Spark SQL permite la creación de DataFrames desde varios formatos de archivos, como CSV, JSON y Parquet. Puedes cargar estos archivos directamente en un DataFrame utilizando la API de Spark SQL. También se pueden usar otros formatos de archivo.
- **Desde tablas Hive**: Puedes crear DataFrames a partir de tablas existentes en Hive, aprovechando el metastore de Hive.
- **Otras fuentes**: Spark puede leer datos de diversas fuentes incluyendo bases de datos relacionales mediante JDBC, NoSQL, ORC, y otros sistemas de almacenamiento.


### 1. Cargar datos desde un RDD:
Para convertir un RDD en un DataFrame, se utiliza la función `toDF()` o `createDataFrame()`.

- **toDF()**: Infiere el esquema del DataFrame a partir del RDD, normalmente usado con una tupla o lista en Python.  https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.toDF.html (similar, la original no está documentada en https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.html)
- **createDataFrame()**: Permite especificar explícitamente el esquema (`StructType`) del DataFrame.  https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.SparkSession.createDataFrame.html

In [3]:
# Inicializamos sesión

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("RDDtoDF").getOrCreate()


In [3]:
import sys
print("Python version: ", sys.version)

Python version:  3.10.16 | packaged by Anaconda, Inc. | (main, Dec 11 2024, 16:19:12) [MSC v.1929 64 bit (AMD64)]


In [4]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
print("Spark version: ", sc.version)


Spark version:  3.5.4


In [4]:
## Ejemplo con toDF():
rdd = spark.sparkContext.parallelize([("Alice", 34), ("Bob", 23)])
df = rdd.toDF(["name", "age"])
df.show()

+-----+---+
| name|age|
+-----+---+
|Alice| 34|
|  Bob| 23|
+-----+---+



In [5]:
## Ejemplo con createDataFrame() (Python):
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

schema = StructType([
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True)
])

rdd = spark.sparkContext.parallelize([("Alice", 34), ("Bob", 23)])
df = spark.createDataFrame(rdd, schema)
df.show()

+-----+---+
| name|age|
+-----+---+
|Alice| 34|
|  Bob| 23|
+-----+---+



### 2. Cargar datos desde ficheros CSV:

#### Sintaxis:
Se utiliza `spark.read.csv()`. Se pueden especificar opciones como `header` para indicar si el archivo tiene encabezado e `inferSchema` para que Spark infiera los tipos de datos.

- `header` indica si la primera línea del archivo CSV contiene los nombres de las columnas.
- `inferSchema` permite a Spark determinar automáticamente los tipos de datos de cada columna.

https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameReader.csv.html

#### Consideraciones
- **Rutas de archivos**: Asegúrate de proporcionar las rutas correctas a tus archivos CSV y Parquet.
- **Esquema**: Si no se utiliza `inferSchema` al leer archivos CSV, el esquema del DataFrame debe especificarse explícitamente.
- **DataFrames**: Los DataFrames proporcionan una forma de procesar y analizar datos estructurados. A diferencia de los RDDs, los DataFrames están basados en un esquema, es decir, conocen los nombres y tipos de las columnas de un conjunto de datos.

In [ ]:
# Desde un fichero CSV
df=spark.read.option("header", "true").option("inferSchema", "true").csv("c:/BDASpark/olive.csv")
df.show()


### 3. Cargar datos desde ficheros Parquet:
Se utiliza `spark.read.parquet()`.  
https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameReader.parquet.html  

#### Consideraciones
- **Esquema**: El esquema se almacena en el mismo archivo.

In [ ]:
# Desde un único fichero parquet
df=spark.read.parquet("c:/BDASpark/palm.parquet")
df.show()


In [ ]:
# Desde un directorio con múltiples archivos
df = spark.read.parquet("c:/BDASpark/olive.parquet")
df.show()


# 2. API de DataFrames:
Para realizar transformaciones en un DataFrame en Spark con Python, se utilizan diversas funciones que permiten modificar, seleccionar, o agregar datos. Aquí te presento las sintaxis y ejemplos de uso de algunas de las transformaciones más comunes:


## Transformaciones:
- **select**: Permite seleccionar columnas específicas de un DataFrame.
- **filter**: Permite filtrar filas basadas en una condición.
- **withColumn**: Permite añadir nuevas columnas o modificar las existentes.
- **Otras transformaciones**: El API incluye otras transformaciones para manipular los datos como groupBy, sort y join. También permite crear funciones definidas por el usuario para manipulación personalizada de datos.

## Acciones:
- **show**: Muestra las primeras filas de un DataFrame.
- **count**: Cuenta el número de filas en un DataFrame.
- **collect**: Retorna todos los elementos de un DataFrame al driver (cuidado con el uso en grandes datasets).
- **Otras acciones**: Incluyen take, takeSample y describe para obtener información y estadísticas sobre los DataFrames.

## Consideraciones:

- **Inmutabilidad**: Los DataFrames son inmutables; cada transformación crea un nuevo DataFrame.
- **show()**: La función `show()` se utiliza para mostrar una muestra de los datos resultantes tras una transformación.
- **Importaciones**: Algunas funciones requieren importaciones adicionales desde `pyspark.sql.functions`, como `col`, `lit`, `expr`, `avg`, `count`, etc.
- **Expresiones SQL**: Puedes usar expresiones SQL con `expr()` y `selectExpr()` para transformaciones más complejas.
- **Columnas**: Las columnas se pueden referenciar usando su nombre como string, usando la notación de corchetes sobre el DataFrame o con la función `col()`.

### 1. select():
Se utiliza para seleccionar un subconjunto de columnas de un DataFrame. También se puede usar `selectExpr()` para seleccionar columnas con expresiones SQL.
https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.select.html

In [ ]:
df = spark.createDataFrame([(2, "Alice"), (5, "Bob")], schema=["age", "name"])

df.select("name", "age").show() 
df.select('*').show()
df.select(df.name, (df.age + 10).alias('age')).show()

# Variante con expresiones SQL
df.selectExpr("name", "age", "age * 2 as double_age").show()

### 2. filter() o where():
Se utiliza para filtrar filas basadas en una condición. `filter()` y `where()` son sinónimos.
https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.filter.html

In [ ]:
df.filter(df["age"] > 30).show()
df.where(df["age"] > 30).show()

### 3. withColumn():
Se utiliza para añadir una nueva columna o reemplazar una existente. La función `lit()` crea una columna con un valor literal y `expr()` permite usar expresiones SQL.
https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.withColumn.html

In [ ]:
from pyspark.sql.functions import lit, expr

df.withColumn("age_plus_10", df["age"] + 10).show()
df.withColumn("is_adult", lit(True)).show()
df.withColumn("age_times_2", expr("age * 2")).show()

### 4. withColumnRenamed():
Se utiliza para renombrar una columna existente.  
https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.withColumnRenamed.html

In [ ]:
df.withColumnRenamed("age", "years").show()

### 5. groupBy():
Se utiliza para agrupar filas con valores iguales en una columna y realizar operaciones de agregación. Se combina con funciones de agregación como `count()`, `sum()`, `avg()`, `min()`, `max()`.  
https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.groupBy.html

In [8]:
df.groupBy("occupation").agg({"age": "avg", "*": "count"}).show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `occupation` cannot be resolved. Did you mean one of the following? [`age`, `name`].;
'Aggregate ['occupation], ['occupation, count(1) AS count(1)#101L, avg(age#14) AS avg(age)#103]
+- LogicalRDD [name#13, age#14], false


### 6. sort() o orderBy():
Se utiliza para ordenar las filas del DataFrame. `sort()` y `orderBy()` son equivalentes y pueden usar el orden ascendente (`asc`) o descendente (`desc`).  
https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.sort.html  
https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.orderBy.html

In [ ]:
df.sort("age").show()
df.sort(df["age"].desc()).show()
df.orderBy("name").show()
df.orderBy(df["name"].desc()).show()

### 7. drop():
Se utiliza para eliminar una o varias columnas del DataFrame.  
https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.drop.html

In [ ]:
df.drop("occupation").show()

### 8. distinct():
Se utiliza para eliminar las filas duplicadas del DataFrame.  
https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.distinct.html

In [ ]:
df.distinct().show()


### 9. join():
Se utiliza para combinar dos DataFrames basados en una o varias columnas en común. Se puede especificar el tipo de join: 'inner', 'outer', 'left_outer', 'right_outer', o 'leftsemi'.  
https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.join.html

In [ ]:
df1.join(df2, df1["id"] == df2["id"], "inner").show()
df1.join(df2, df1["id"] == df2["id"], "left_outer").show()

### 10. union() o unionAll():
Se utiliza para combinar dos DataFrames con las mismas columnas. `union()` elimina duplicados, `unionAll()` no.  
    https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.union.html
y https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.unionAll.html

In [ ]:
df1.union(df2).show()
df1.unionAll(df2).show()


### 11. map():
Se utiliza para aplicar una función a cada fila del DataFrame, convirtiéndolo a RDD.  
La función map() se aplica a RDDs (Resilient Distributed Datasets), no directamente a DataFrames. Para usar map en un DataFrame, primero debes convertirlo a un RDD usando df.rdd.   
https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.map.html


In [ ]:
df = spark.createDataFrame([(2, "Alice"), (5, "Bob")], schema=["age", "name"])
df.rdd.map(lambda row: (row.name, row.age * 2)).toDF(["name", "double_age"]).show()